## Pré Processamento dos Dados

### Bibliotecas e base de dados

In [45]:
import pandas as pd

In [46]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv("https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
                         sep=';',
                         decimal=',',
                         dtype = dtype)

df_eventos.head(5)

,Protocolo_S2iD,Nome_Municipio,Sigla_UF,regiao,Data_Registro,Data_Evento,codigo_ibge,Setores Censitários,Status,DH_Descricao,...,Hab,Area,Ano,PIB,Categoria,Grupo,Subgrupo,Tipo,Subtipo,COBRADE
0,SC-F-4200903-12200-20141226,Angelina,SC,Sul,2015-01-02 00:00:00,2015-01-02,4200903,NaN,Reconhecido,Moradores da comunidade que ficarem desprovido...,...,5358.0,499.998,2015.0,103375.0,Natural,Hidrológico,Enxurradas,NaN,NaN,12200
1,MS-F-5000708-12200-20141222,Anastácio,MS,Centro-oeste,2015-01-05 00:00:00,2015-01-05,5000708,NaN,Reconhecido,Ficaram prejudicados um total de 800 famílias ...,...,24114.0,2913.177,2015.0,394893.0,Natural,Hidrológico,Enxurradas,NaN,NaN,12200
2,RS-F-4301925-12200-20150101,Barra do Rio Azul,RS,Sul,2015-01-06 00:00:00,2015-01-06,4301925,NaN,Não reconhecido,NaN,...,1696.0,146.995,2015.0,42201.0,Natural,Hidrológico,Enxurradas,NaN,NaN,12200
3,RS-F-4311601-12200-20141231,Liberato Salzano,RS,Sul,2015-01-07 00:00:00,2015-01-07,4311601,NaN,Reconhecido,NaN,...,4781.0,245.627,2015.0,116822.0,Natural,Hidrológico,Enxurradas,NaN,NaN,12200
4,BA-F-2931608-12200-20141216,Teolândia,BA,Nordeste,2015-01-07 00:00:00,2015-01-07,2931608,NaN,Reconhecido,05 pessoas deram entrada no hospital municipal...,...,15332.0,289.782,2015.0,112279.0,Natural,Hidrológico,Enxurradas,NaN,NaN,12200


### Tipo das variáveis e dados ausentes

In [47]:
df_eventos.dtypes.reset_index().rename(columns={0: '.dtype', 'index': 'Variável'})

,Variável,.dtype
0,Protocolo_S2iD,object
1,Nome_Municipio,object
2,Sigla_UF,object
3,regiao,object
4,Data_Registro,object
...,...,...
72,Grupo,object
73,Subgrupo,object
74,Tipo,object
75,Subtipo,object


In [48]:
#Verificar se há protocolos duplicados

df_eventos["Protocolo_S2iD"].duplicated().sum()

#Não há protocolos duplicados

np.int64(0)

In [49]:
# Excluir variáveis que não serão adicionadas ao modelo

df_eventos = df_eventos.drop(['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao', 'Data_Registro', 'Data_Evento', 'codigo_ibge',
                'Setores Censitários', 'COBRADE', "Ano"], axis=1)

### Variáveis com dados ausentes

In [50]:
df_na_counts = df_eventos.isna().sum().reset_index().rename(columns={0: 'Qtde. dados ausentes', 'index': 'Variável'})
df_na_counts['% de dados ausentes'] = (df_na_counts['Qtde. dados ausentes'] / len(df_eventos)) * 100
df_na_counts = df_na_counts.sort_values(by='% de dados ausentes', ascending=False)
df_na_counts

,Variável,Qtde. dados ausentes,% de dados ausentes
32,DA_Polui/cont do ar,18213,99.275046
33,DA_Polui/cont do solo,17991,98.064973
31,DA_Polui/cont da água,17371,94.685490
35,DA_Incêndi parques/APA's/APP's,17009,92.712308
34,DA_Dimi/exauri hídrico,16668,90.853592
...,...,...,...
6,DH_DESALOJADOS,0,0.000000
5,DH_DESABRIGADOS,0,0.000000
4,DH_ENFERMOS,0,0.000000
3,DH_FERIDOS,0,0.000000


In [51]:
#Excluir colunas que tenham mais do que 5% de dados ausentes
colunas_mantidas = df_na_counts[df_na_counts['% de dados ausentes'] < 5]['Variável']
df_eventos = df_eventos[colunas_mantidas]
print(df_eventos.columns)

Index(['Subgrupo', 'PEPL_Distrib combustíveis(R$)', 'PEPL_total_publico',
       'PEPL_Ensino (R$)', 'PEPL_Segurança pública (R$)', 'DH_DESAPARECIDOS',
       'PEPL_Tran loc/reg/l_curso (R$)', 'PEPR_Agricultura (R$)',
       'PEPL_distrib energia (R$)', 'PEPL_Sis cont pragas (R$)',
       'PEPL_Telecomunicações (R$)', 'PEPR_Indústria (R$)',
       'PEPR_Pecuária (R$)', 'PEPL_sist de esgotos sanit(R$)',
       'PEPR_Comércio (R$)', 'PEPR_Serviços (R$)', 'PEPR_total_privado',
       'PE_PLePR', 'DH_MORTOS', 'DensidadePop', 'Hab', 'Area', 'Categoria',
       'Grupo', 'PEPL_Sis limp e rec lixo (R$)',
       'PEPL_Assis_méd e emergên(R$)', 'PEPL_Abast de água pot(R$)',
       'DM_Inst Serviços Destruidas', 'DH_total_danos_humanos',
       'DM_Uni Habita Danificadas', 'DM_Uni Habita Destruidas',
       'DM_Uni Habita Valor', 'DM_Inst Saúde Danificadas',
       'DM_Inst Saúde Destruidas', 'DM_Inst Saúde Valor',
       'DM_Inst Ensino Danificadas', 'DM_Inst Ensino Destruidas',
       'DM_Inst 

### Encoding das variáveis categóricas

In [52]:
df_eventos = pd.get_dummies(df_eventos, columns=['Subgrupo', 'Categoria', 'Grupo'], drop_first=False)
df_eventos.head(3)

,PEPL_Distrib combustíveis(R$),PEPL_total_publico,PEPL_Ensino (R$),PEPL_Segurança pública (R$),DH_DESAPARECIDOS,PEPL_Tran loc/reg/l_curso (R$),PEPR_Agricultura (R$),PEPL_distrib energia (R$),PEPL_Sis cont pragas (R$),PEPL_Telecomunicações (R$),...,Categoria_Tecnológico,Grupo_Biológico,Grupo_Climatológico,Grupo_Desastres Relacionados a Incêndios Urbanos,Grupo_Desastres Relacionados a Produtos Perigosos,Grupo_Desastres relacionados a obras civis,Grupo_Desastres relacionados a transporte de passageiros e cargas não perigosas,Grupo_Geológico,Grupo_Hidrológico,Grupo_Meteorológico
0,0.0,0.000000e+00,0.0,0.0,0,0.000000e+00,0.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,0.0,2.332357e+06,0.0,0.0,0,2.332357e+06,326156.773661,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,0.0,0.000000e+00,0.0,0.0,0,0.000000e+00,0.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
